In [ ]:
import pandas as pd

df_ac = pd.read_csv('../Data/Account.csv')
unique_account = set(df_ac['crm_Account_Account'])
df_ac.drop('crm_Account_Hoofd_NaCe_Code', axis=1, inplace=True)
df_ac.drop_duplicates(keep='first', inplace=True)
df_ac.to_csv('../Data/Account.csv' ,index=False)


df_all = pd.read_csv('../Data/Afspraak alle.csv')
unique_afspraak = set(df_all['crm_Afspraak_ALLE_Afspraak'])

df_cam = pd.read_csv('../Data/Campagne.csv')
unique_campaign = set(df_cam['crm_Campagne_Campagne'])

df_cam["crm_Campagne_Einddatum"] = pd.to_datetime(df_cam["crm_Campagne_Einddatum"]).dt.date
df_cam["crm_Campagne_Startdatum"] = pd.to_datetime(df_cam["crm_Campagne_Startdatum"]).dt.date

df_cam.to_csv('../Data/Campagne.csv', index=False)

df_mail = pd.read_csv("../Data/CDI mailing.csv")
unique_mail = set(df_mail["crm_CDI_Mailing_Mailing"])

df_per = pd.read_csv('../Data/Persoon.csv')
unique_persoon = set(df_per["crm_Persoon_persoon"])
df_per.drop(['crm_Persoon_Mail_regio_Antwerpen_Waasland',
              'crm_Persoon_Mail_regio_Brussel_Hoofdstedelijk_Gewest',
              'crm_Persoon_Mail_regio_Limburg',
              'crm_Persoon_Mail_regio_Mechelen_Kempen',
              'crm_Persoon_Mail_regio_Oost_Vlaanderen',
              'crm_Persoon_Mail_regio_Vlaams_Brabant',
              'crm_Persoon_Mail_regio_Voka_nationaal',
              'crm_Persoon_Mail_regio_West_Vlaanderen'
              ], axis=1, inplace=True)
df_per.drop(['crm_Persoon_Mail_thema_duurzaamheid',
              'crm_Persoon_Mail_thema_financieel_fiscaal',
              'crm_Persoon_Mail_thema_innovatie',
              'crm_Persoon_Mail_thema_internationaal_ondernemen',
              'crm_Persoon_Mail_thema_mobiliteit',
              'crm_Persoon_Mail_thema_omgeving',
              'crm_Persoon_Mail_thema_sales_marketing_communicatie',
              'crm_Persoon_Mail_thema_strategie_en_algemeen_management',
              'crm_Persoon_Mail_thema_talent',
              'crm_Persoon_Mail_thema_welzijn',
              ], axis=1, inplace=True)
df_per.drop(['crm_Persoon_Mail_type_Bevraging',
              'crm_Persoon_Mail_type_communities_en_projecten',
              'crm_Persoon_Mail_type_netwerkevenementen',
              'crm_Persoon_Mail_type_nieuwsbrieven',
              'crm_Persoon_Mail_type_opleidingen',
              'crm_Persoon_Mail_type_persberichten_belangrijke_meldingen',
              ], axis=1, inplace=True)   
df_per.drop('crm_Persoon_Web_Login', axis=1, inplace=True)
df_per.to_csv('../Data/Persoon.csv', index=False)        

df_con = pd.read_csv('../Data/Contact.csv')
unique_contact = set(df_con['crm_Contact_Contactpersoon'])

df_func = pd.read_csv("../Data/Functie.csv")
unique_functie = set(df_func["crm_Functie_Functie"])

df_geb = pd.read_csv("../Data/Gebruikers.csv")
unique_gebruiker = set(df_geb['crm_Gebruikers_CRM_User_ID'])

df_insc = pd.read_csv("../Data/Inschrijving.csv")
unique_inschrijving = set(df_insc["crm_Inschrijving_Inschrijving"])

df_insc = df_insc[df_insc['crm_Inschrijving_Contactfiche'].isin(unique_contact)]
df_insc = df_insc[df_insc["crm_Inschrijving_Bron"].isin(unique_campaign)]

df_insc.drop_duplicates(subset='crm_Inschrijving_Inschrijving', inplace=True, keep='first')

df_insc.to_csv("../Data/Inschrijving.csv", index=False)

df_ses = pd.read_csv("../Data/Sessie.csv")
unique_sessie = set(df_ses["crm_Sessie_Sessie"])

df_acti = pd.read_csv('../Data/Activiteitscode.csv')
unique_activiteit = set(df_acti['crm_ActiviteitsCode_Activiteitscode'])


In [ ]:
df_con = pd.read_csv('../Data/Contact.csv')
unique_contact = set(df_con['crm_Contact_Contactpersoon'])

In [ ]:
df_aac = pd.read_csv('../Data/Account activiteitscode.csv')


df_aac.drop_duplicates(keep='first', inplace=True)
df_aac.dropna(inplace=True)

df_aac = df_aac[df_aac['crm_Account_ActiviteitsCode_Account'].isin(unique_account)]

df_aac.to_csv('../Data/Account activiteitscode.csv', index=False)


In [ ]:
df_fd = pd.read_csv("../Data/Account financiële data.csv")
df_fd = df_fd[df_fd['crm_FinancieleData_OndernemingID'].isin(unique_account)]
df_fd = df_fd.drop_duplicates(subset=['crm_FinancieleData_OndernemingID', 'crm_FinancieleData_Boekjaar'], keep='first')

df_fd.to_csv("../Data/Account financiële data.csv", index=False)

In [ ]:
df_pa = pd.read_csv("../Data/cdi pageviews.csv")
mean_duration = df_pa["crm_CDI_PageView_Duration"].mean()

df_pa.drop('crm_CDI_PageView_Anonymous_Visitor', inplace=True, axis=1)
df_pa.drop('crm_CDI_PageView_Url', inplace=True, axis=1)
df_pa.drop('crm_CDI_PageView_Web_Content', inplace=True, axis=1)

df_pa.drop_duplicates(keep='first', inplace=True)

df_pa = df_pa[df_pa['crm_CDI_PageView_Contact'].isin(unique_contact)]
df_pa = df_pa[df_pa['crm_CDI_PageView_Campaign'].isin(unique_campaign)]

df_pa["crm_CDI_PageView_Time"] = pd.to_datetime(df_pa["crm_CDI_PageView_Time"].str.replace('(UTC)', ''))
df_pa["crm_CDI_PageView_Viewed_On"] = pd.to_datetime(df_pa["crm_CDI_PageView_Viewed_On"])
df_pa["crm_CDI_PageView_Aangemaakt_op"] = pd.to_datetime(df_pa["crm_CDI_PageView_Aangemaakt_op"])
df_pa["crm_CDI_PageView_Gewijzigd_op"] = pd.to_datetime(df_pa["crm_CDI_PageView_Gewijzigd_op"])


df_pa["crm_CDI_PageView_Duration"].fillna(mean_duration, inplace=True)

df_pa.to_csv("../Data/cdi pageviews.csv", index=False)

df_pa.head()

In [ ]:
df_vi = pd.read_csv("../Data/CDI visits.csv")


# df_vi.drop(["crm_CDI_Visit_Browser",
#             "crm_CDI_Visit_Campagne_Code",
#             "crm_CDI_Visit_Referring_Host",
#             "crm_CDI_Visit_Bounce",
#             "crm_CDI_Visit_Adobe_Reader",
#             "crm_CDI_Visit_containssocialprofile",
#             "crm_CDI_Visit_IP_Company",
#             "crm_CDI_Visit_Entry_Page",
#             "crm_CDI_Visit_Exit_Page",
#             "crm_CDI_Visit_Referrer",
#             "crm_CDI_Visit_Total_Pages",
#             "crm_CDI_Visit_Keywords" ], axis=1, inplace=True) 

df_vi.drop_duplicates(subset=["crm_CDI_Visit_Visit","crm_CDI_Visit_Contact" ], keep="first", inplace=True)

df_vi = df_vi[df_vi["crm_CDI_Visit_Contact"].isin(unique_contact)]
df_vi = df_vi[df_vi["crm_CDI_Visit_Email_Send"].isin(unique_mail)]
df_vi = df_vi[df_vi["crm_CDI_Visit_Campaign"].isin(unique_campaign)]

df_vi["crm_CDI_Visit_Time"] = pd.to_datetime(df_vi["crm_CDI_Visit_Time"].str.replace('(UTC)', ''))
df_vi["crm_CDI_Visit_Aangemaakt_op"] = pd.to_datetime(df_vi["crm_CDI_Visit_Aangemaakt_op"]) #, format="%d-%m-%Y %H:%M:%S"
df_vi["crm_CDI_Visit_Gewijzigd_op"] = pd.to_datetime(df_vi["crm_CDI_Visit_Gewijzigd_op"]) # , format="%d-%m-%Y %H:%M:%S"
df_vi["crm_CDI_Visit_Ended_On"] = pd.to_datetime(df_vi["crm_CDI_Visit_Ended_On"]) # , format="%d-%m-%Y %H:%M:%S"
df_vi["crm_CDI_Visit_Started_On"] = pd.to_datetime(df_vi["crm_CDI_Visit_Started_On"]) # , format="%d-%m-%Y %H:%M:%S"

df_vi.to_csv("../Data/CDI visits.csv", index=False)


In [ ]:
df_con = df_con[df_con['crm_Contact_Account'].isin(unique_account)]
df_con = df_con[df_con['crm_Contact_Persoon_ID'].isin(unique_persoon)]
df_con.drop_duplicates('crm_Contact_Contactpersoon', keep='first', inplace=True)

df_con.to_csv('../Data/Contact.csv', index=False)

In [ ]:
df_gc = pd.read_csv('../Data/Afspraak_account_gelinkt_cleaned.csv')

df_gc = df_gc[df_gc['crm_Afspraak_ACCOUNT_GELINKT_Afspraak'].isin(unique_afspraak)]
df_gc.drop_duplicates('crm_Afspraak_ACCOUNT_GELINKT_Afspraak',keep='first', inplace=True)

df_gc.to_csv('../Data/Afspraak_account_gelinkt_cleaned.csv', index=False)

In [ ]:
df_cc = pd.read_csv("../Data/Afspraak betreft contact_cleaned.csv")

df_cc = df_cc[df_cc['crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id'].isin(unique_contact)]
df_cc = df_cc[df_cc['crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak'].isin(unique_afspraak)]
df_cc.drop_duplicates(['crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id', 'crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak'],keep='first' ,inplace=True )

df_cc.to_csv("../Data/Afspraak betreft contact_cleaned.csv", index=False)

In [150]:
df_con = pd.read_csv('../Data/Contact.csv')
unique_contact = set(df_con['crm_Contact_Contactpersoon'])

df_acti = pd.read_csv('../Data/Activiteitscode.csv')
unique_activiteit = set(df_acti["crm_ActiviteitsCode_Activiteitscode"])

df_vc = pd.read_csv("../Data/Activiteit vereist contact.csv")

# df_vc = df_vc[df_vc["crm_ActiviteitVereistContact_ActivityId"].isin(unique_activiteit)]
df_vc = df_vc[df_vc["crm_ActiviteitVereistContact_ReqAttendee"].isin(unique_contact)]
df_vc.drop_duplicates(['crm_ActiviteitVereistContact_ReqAttendee', 'crm_ActiviteitVereistContact_ActivityId'], keep="first", inplace=True)

# df_vc.info()

df_vc.to_csv("../Data/Activiteit vereist contact.csv", index=False)

In [ ]:
df_acc = pd.read_csv("../Data/Afspraak betreft account_cleaned.csv")

df_acc = df_acc[df_acc['crm_Afspraak_BETREFT_ACCOUNT_Betreft_id'].isin(unique_account)]
df_acc = df_acc[df_acc['crm_Afspraak_BETREFT_ACCOUNT_Afspraak'].isin(unique_afspraak)]
df_acc.drop_duplicates(keep='first', inplace=True)

df_acc.to_csv("../Data/Afspraak betreft account_cleaned.csv", index=False)

In [ ]:
df_cf = pd.read_csv("../Data/Contact functie.csv")

df_cf = df_cf[df_cf['crm_ContactFunctie_Functie'].isin(unique_functie)]
df_cf = df_cf[df_cf['crm_ContactFunctie_Contactpersoon'].isin(unique_contact)]
df_cf.drop_duplicates(keep='first', inplace=True)

df_cf.to_csv("../Data/Contact functie.csv", index=False)

In [ ]:
df_sent = pd.read_csv("../Data/CDI sent email clicks.csv")

df_sent = df_sent[df_sent['crm_CDI_SentEmail_kliks_Contact'].isin(unique_contact)]
df_sent = df_sent[df_sent['crm_CDI_SentEmail_kliks_E_mail_versturen'].isin(unique_mail)]
df_sent.drop_duplicates(keep='first', inplace=True)

df_sent.to_csv("../Data/CDI sent email clicks.csv", index=False)

In [ ]:
df_info = pd.read_csv("../Data/Info en klachten.csv")

df_info = df_info[df_info["crm_Info_en_Klachten_Eigenaar"].isin(unique_gebruiker)]
df_info = df_info[df_info["crm_Info_en_Klachten_Account"].isin(unique_account)]
df_info.drop_duplicates(keep="first", inplace=True)

df_info.to_csv("../Data/Info en klachten.csv", index=False)

In [5]:
import pandas as pd
df_lid = pd.read_csv("../Data/Lidmaatschap.csv")

df_ac = pd.read_csv('../Data/Account.csv')
unique_account = set(df_ac['crm_Account_Account'])

df_lid["crm_Lidmaatschap_Datum_Opzeg"] = pd.to_datetime(df_lid["crm_Lidmaatschap_Datum_Opzeg"] , format="%d-%m-%Y" ) #
df_lid["crm_Lidmaatschap_Startdatum"] = pd.to_datetime(df_lid["crm_Lidmaatschap_Startdatum"]) #, format="%d-%m-%Y" 
df_lid = df_lid[df_lid["crm_Lidmaatschap_Onderneming"].isin(unique_account)]

df_lid.drop_duplicates(keep="first", inplace=True)

# df_lid.to_csv("../Data/Lidmaatschap.csv", index=False)
df_lid.head()

,crm_Lidmaatschap_Datum_Opzeg,crm_Lidmaatschap_Lidmaatschap,crm_Lidmaatschap_Onderneming,crm_Lidmaatschap_Reden_Aangroei,crm_Lidmaatschap_Reden_Verloop,crm_Lidmaatschap_Startdatum
0,NaT,0014C090-3F02-E611-8668-005056B06EC4,E7726D1E-C80A-E511-ABE8-005056B06EB4,Actieve werving: koude prospectie,NaN,2016-04-11
1,NaT,0023960B-C39F-E311-B1AE-005056B06EC4,D64157FA-9F68-E111-B43A-00505680000A,Lead van een collega nalv contact of deelname,NaN,2014-02-12
2,NaT,00296269-096F-E111-B43A-00505680000A,19EB325F-C368-E111-B43A-00505680000A,NaN,NaN,2014-01-01
3,NaT,0065C4C5-9403-EE11-8F6E-6045BD895D4D,E31B5630-F6EE-ED11-8849-6045BD895D4D,Actieve werving: koude prospectie,NaN,2023-05-23
4,NaT,0082BAB8-FD6E-E111-B43A-00505680000A,92A753BA-0569-E111-B43A-00505680000A,NaN,NaN,2007-01-01


In [ ]:
df_ses = df_ses[df_ses["crm_Sessie_Campagne"].isin(unique_campaign)]
df_ses.drop_duplicates(keep='first', inplace=True)

df_ses["crm_Sessie_Eind_Datum_Tijd"] = pd.to_datetime(df_ses["crm_Sessie_Eind_Datum_Tijd"], format="%d-%m-%Y %H:%M:%S") 
df_ses["crm_Sessie_Start_Datum_Tijd"] = pd.to_datetime(df_ses["crm_Sessie_Start_Datum_Tijd"], format="%d-%m-%Y %H:%M:%S") 


df_ses.to_csv("../Data/Sessie.csv", index=False)

In [17]:
df_insc = pd.read_csv("../Data/Inschrijving.csv")
unique_inschrijving = set(df_insc["crm_Inschrijving_Inschrijving"])

df_con = pd.read_csv('../Data/Contact.csv')
unique_contact = set(df_con['crm_Contact_Contactpersoon'])



# df_insc.to_csv("../Data/Inschrijving.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363171 entries, 0 to 363170
Data columns (total 6 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   crm_Inschrijving_Aanwezig_Afwezig    363171 non-null  object        
 1   crm_Inschrijving_Bron                363139 non-null  object        
 2   crm_Inschrijving_Contactfiche        363171 non-null  object        
 3   crm_Inschrijving_Datum_inschrijving  363171 non-null  datetime64[ns]
 4   crm_Inschrijving_Inschrijving        363171 non-null  object        
 5   crm_Inschrijving_Facturatie_Bedrag   363171 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 16.6+ MB


In [50]:
df_insc = pd.read_csv("../Data/Inschrijving.csv")
df_ses_in = pd.read_csv("../Data/Sessie inschrijving.csv")

if 'crm_Inschrijving_Campagne' not in df_insc.columns:
    # # Merge Inschrijving and Sessie_Inschrijving on Inschrijving_ID
    df_merged = pd.merge(df_insc, df_ses_in, how='inner',  left_on='crm_Inschrijving_Inschrijving', right_on='crm_SessieInschrijving_Inschrijving')

    # # Merge with Sessie on Sessie_ID
    df_merged = pd.merge(df_merged, df_ses, how='inner', left_on='crm_SessieInschrijving_Sessie', right_on='crm_Sessie_Sessie') #crm_Sessie_Sessie

    # # Merge with Campagne on Campagne_ID
    df_merged = pd.merge(df_merged, df_cam, how='inner', left_on='crm_Sessie_Campagne', right_on='crm_Campagne_Campagne')


    df_insc = df_merged[['crm_Inschrijving_Aanwezig_Afwezig',
                        'crm_Inschrijving_Bron',
                        'crm_Inschrijving_Contactfiche',
                        'crm_Inschrijving_Datum_inschrijving',
                        'crm_Inschrijving_Inschrijving',
                        'crm_Inschrijving_Facturatie_Bedrag',
                        'crm_Campagne_Campagne',
                        'crm_Campagne_Naam'
                        ]].drop_duplicates()

    df_insc.rename(columns={'crm_Campagne_Campagne':"crm_Inschrijving_Campagne",
                            'crm_Campagne_Naam':'crm_Inschrijving_Campagne_Naam_'}, inplace=True)


df_insc = df_insc[df_insc['crm_Inschrijving_Contactfiche'].isin(unique_contact)]
df_insc = df_insc[df_insc["crm_Inschrijving_Campagne"].isin(unique_campaign)]

df_insc["crm_Inschrijving_Datum_inschrijving"] = pd.to_datetime(df_insc["crm_Inschrijving_Datum_inschrijving"])

df_insc.drop_duplicates(subset='crm_Inschrijving_Inschrijving', inplace=True, keep='first')

# df_insc.info()
df_insc.to_csv("../Data/Inschrijving.csv", index=False)


IndentationError: expected an indented block after 'if' statement on line 10 (4275037747.py, line 13)

In [ ]:
df_ses_in = pd.read_csv("../Data/Sessie inschrijving.csv")

df_ses_in = df_ses_in[df_ses_in["crm_SessieInschrijving_Sessie"].isin(unique_sessie)]
df_ses_in = df_ses_in[df_ses_in["crm_SessieInschrijving_Inschrijving"].isin(unique_inschrijving)]
df_ses_in.drop_duplicates(keep="first", inplace=True)


df_ses_in.to_csv("../Data/Sessie inschrijving.csv", index=False)